clean up

In [0]:
%sql
DROP TABLE IF EXISTS flights_parquet_delta

In [0]:
%fs

rm -r /mnt/filesystem/delta_01/flights_parquet_delta

res0: Boolean = true

### Delta transaction log

In [0]:
df = spark.read.format('csv')\
  .option('header', True)\
  .option('inferSchema', True)\
  .load('dbfs:/databricks-datasets/flights/departuredelays.csv')\
  .sort('date')\
  .limit(2)

df.write\
  .format('delta')\
  .mode('overwrite')\
  .saveAsTable(name='flights_parquet_delta', path='dbfs:/mnt/filesystem/delta_01/flights_parquet_delta')

In [0]:
%sql

SELECT * FROM flights_parquet_delta

date,delay,distance,origin,destination
1010005,-8,2024,LAX,PBI
1010010,-6,1980,SEA,CLT


In [0]:
%fs

ls /mnt/filesystem/delta_01/flights_parquet_delta

path,name,size
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/,_delta_log/,0
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/part-00000-8a4909b1-5323-4cdc-a52f-d1cdf9aef254-c000.snappy.parquet,part-00000-8a4909b1-5323-4cdc-a52f-d1cdf9aef254-c000.snappy.parquet,1311


In [0]:
%fs

ls /mnt/filesystem/delta_01/flights_parquet_delta/_delta_log

path,name,size
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,1649
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/__tmp_path_dir/,__tmp_path_dir/,0


In [0]:
%fs

head /mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/00000000000000000000.json

{"commitInfo":{"timestamp":1620813775191,"userId":"6531383946989840","userName":"pawel.mitrus@lingarogroup.com","operation":"CREATE OR REPLACE TABLE AS SELECT","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{}"},"notebook":{"notebookId":"1646022116006925"},"clusterId":"0425-045756-dying623","isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numOutputBytes":"1311","numOutputRows":"2"}}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"64fce33f-e0d7-451c-aed8-299deb37f82f","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"date\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"delay\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"distance\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"origin\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"destination\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1620813770466}}
{"add":{"path":"part-00000-8a4909b1-5323-4cdc-a52f-d1cdf9aef254-c000.snappy.parquet","partitionValues":{},"size":1311,"modificationTime":1620813774000,"dataChange":true,"stats":"{\"numRecords\":2,\"minValues\":{\"date\":1010005,\"delay\":-8,\"distance\":1980,\"origin\":\"LAX\",\"destination\":\"CLT\"},\"maxValues\":{\"date\":1010010,\"delay\":-6,\"distance\":2024,\"origin\":\"SEA\",\"destination\":\"PBI\"},\"nullCount\":{\"date\":0,\"delay\":0,\"distance\":0,\"origin\":0,\"destination\":0}}"}}

### UPDATE

In [0]:
%sql

UPDATE flights_parquet_delta
SET delay = 0
WHERE date = 1010005

num_affected_rows
1


In [0]:
%sql

SELECT * FROM flights_parquet_delta

date,delay,distance,origin,destination
1010005,0,2024,LAX,PBI
1010010,-6,1980,SEA,CLT


In [0]:
%fs

ls /mnt/filesystem/delta_01/flights_parquet_delta/_delta_log

path,name,size
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,1649
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/00000000000000000001.crc,00000000000000000001.crc,89
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/00000000000000000001.json,00000000000000000001.json,1247
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/__tmp_path_dir/,__tmp_path_dir/,0


In [0]:
%fs

head /mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/00000000000000000001.json

{"commitInfo":{"timestamp":1620814059420,"userId":"6531383946989840","userName":"pawel.mitrus@lingarogroup.com","operation":"UPDATE","operationParameters":{"predicate":"(date#743 = 1010005)"},"notebook":{"notebookId":"1646022116006925"},"clusterId":"0425-045756-dying623","readVersion":0,"isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numCopiedRows":"1","numAddedChangeFiles":"0","executionTimeMs":"3966","scanTimeMs":"883","numAddedFiles":"1","numUpdatedRows":"1","rewriteTimeMs":"3083"}}}
{"remove":{"path":"part-00000-8a4909b1-5323-4cdc-a52f-d1cdf9aef254-c000.snappy.parquet","deletionTimestamp":1620814056335,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":1311}}
{"add":{"path":"part-00000-a43fdf9f-38e2-4b0d-be89-0084080f5f0b-c000.snappy.parquet","partitionValues":{},"size":1311,"modificationTime":1620814059000,"dataChange":true,"stats":"{\"numRecords\":2,\"minValues\":{\"date\":1010005,\"delay\":-6,\"distance\":1980,\"origin\":\"LAX\",\"destination\":\"CLT\"},\"maxValues\":{\"date\":1010010,\"delay\":0,\"distance\":2024,\"origin\":\"SEA\",\"destination\":\"PBI\"},\"nullCount\":{\"date\":0,\"delay\":0,\"distance\":0,\"origin\":0,\"destination\":0}}"}}

In [0]:
%sql
DESCRIBE HISTORY flights_parquet_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2021-05-12T10:07:39.000+0000,6531383946989840,pawel.mitrus@lingarogroup.com,UPDATE,Map(predicate -> (date#743 = 1010005)),null,List(1646022116006925),0425-045756-dying623,0,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 1, numAddedChangeFiles -> 0, executionTimeMs -> 3966, scanTimeMs -> 883, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 3083)",null
0,2021-05-12T10:02:55.000+0000,6531383946989840,pawel.mitrus@lingarogroup.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(1646022116006925),0425-045756-dying623,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputBytes -> 1311, numOutputRows -> 2)",null


In [0]:
%fs

ls /mnt/filesystem/delta_01/flights_parquet_delta/

path,name,size
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/_delta_log/,_delta_log/,0
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/part-00000-8a4909b1-5323-4cdc-a52f-d1cdf9aef254-c000.snappy.parquet,part-00000-8a4909b1-5323-4cdc-a52f-d1cdf9aef254-c000.snappy.parquet,1311
dbfs:/mnt/filesystem/delta_01/flights_parquet_delta/part-00000-a43fdf9f-38e2-4b0d-be89-0084080f5f0b-c000.snappy.parquet,part-00000-a43fdf9f-38e2-4b0d-be89-0084080f5f0b-c000.snappy.parquet,1311
